# Student Life Safety Search Notebook

## Load libraries and requirements
- run pip libraries once then comment out once installed

In [ ]:
#Run these once to make sure you have libraries installed, then can comment out
#%pip install --upgrade msticpy
#%pip install pandas
#%pip install azure-kusto-data

import ipywidgets as widgets
import msticpy
import time
import json
import pandas as pd

from IPython.display import display, HTML
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
from datetime import datetime, timedelta


msticpy.init_notebook(
    namespace=globals(),
    verbosity=0,
);



## Begin Search - Username and Time Window
- Generate a user interface and selectors to search for our user and time window

In [ ]:
#Create Username Search Field
username = widgets.Text(value='', placeholder='Enter Username')
# Display the dropdown
print("Input Username and Time Search Range")

# Calculate default dates
today = datetime.today()
seven_days_ago = today - timedelta(days=7)

# Create date pickers
start_date_picker = widgets.DatePicker(
    description='Start Date',
    value=seven_days_ago.date()
)
finish_date_picker = widgets.DatePicker(
    description='Finish Date',
    value=today.date()
)

# Display the date pickers
display(username, start_date_picker, finish_date_picker)




## Initialize Connection to Azure Data Explorer
 

In [3]:
# Convert to datetime
start_datetime = datetime.combine(start_date_picker.value, datetime.min.time())
finish_datetime = datetime.combine(finish_date_picker.value, datetime.min.time())

# Add your ADX Cluster Information Here
AAD_TENANT_ID = "Your Azure Tenant ID"
KUSTO_CLUSTER = "Your ADX Cluster base URL"
KUSTO_DATABASE = "Your ADX Database Name"

KCSB = KustoConnectionStringBuilder.with_aad_device_authentication(KUSTO_CLUSTER)
KCSB.authority_id = AAD_TENANT_ID

KUSTO_CLIENT = KustoClient(KCSB)

## Query ADX for Radius Logs: Username->MAC Address
- getting radius logs from an external data lake source as they don't typically need to reside in a SIEM
- this will map our user to specific devices over the time window in question

In [ ]:
radiuskql = f"radius \
| where timestamp > datetime({start_datetime}) \
| where timestamp < datetime({finish_datetime}) \
| where username == '{username.value}' \
| where isnotempty(source_mac) \
| distinct source_mac"

print(radiuskql)

radiusResponse = KUSTO_CLIENT.execute(KUSTO_DATABASE, radiuskql)

macaddresses = dataframe_from_result_table(radiusResponse.primary_results[0])
macaddresses

mac_list = ", ".join(f"'{value}'" for value in macaddresses["source_mac"])
mac_list
print("MAC Address list for the selected user is complete.  Now move on to search for WAP Authentication Events.")

## Connect to Microsoft Sentinel

In [ ]:
#%pip install msticpy[azure]
from msticpy.data.azure import MicrosoftSentinel
microsoft_sentinel = MicrosoftSentinel()
microsoft_sentinel.connect()
print("Successfully connected to Azure Sentinel")

## Search Sentinel for Aruba Wireless matching MAC Addresses
- based on Aruba Analytics and Location Engine (ALE)
- this will apply our user's observed devices across physical wireless access points, including likely distance from decibel levels

In [ ]:
arubakql = f"arubaALE_CL \
| where timestamp_t > datetime({start_datetime}) \
| where timestamp_t < datetime({finish_datetime}) \
| where ['CLIENT_MAC_s'] in ({mac_list})"

qry_prov = QueryProvider("MSSentinel")
qry_prov.connect(WorkspaceConfig())
wireless_results = qry_prov.exec_query(arubakql)
print("Wireless log search complete and loaded")
wireless_results

## Search Sentinel for Building Access Logs
- based on Lenel system access logs

In [ ]:
buildingkql = f"buildingaccess_CL \
| where eventtime_t > datetime({start_datetime}) \
| where eventtime_t < datetime({finish_datetime}) \
| where user_s == '{username.value}'"

qry_prov = QueryProvider("MSSentinel")
qry_prov.connect(WorkspaceConfig())
buildinglogs_results = qry_prov.exec_query(buildingkql)
print("Building log search complete and loaded")
buildinglogs_results

## Retrieve GPS Coordinates of Wireless Devices and Building Access Points
- we utilize a Sentinel watchlist for this storage; many other options may exist

In [ ]:
gpskql = "Watchlist \
| where WatchlistAlias == 'devicegpslookup' \
| where isnotnull(WatchlistItem)"

qry_prov = QueryProvider("MSSentinel")
qry_prov.connect(WorkspaceConfig())
gps_results = qry_prov.exec_query(gpskql)
print("Search Complete and Loaded")
gps_results

## Join GPS data with wireless and building logs
- use panda dataframes to join data and assemble timeline

In [9]:
#Unpack the JSON Object of 'WatchlistItem' into discrete fields within a panda dataframe object
gpsItems = pd.json_normalize(gps_results['WatchlistItem'].apply(json.loads))

#Merge Data for physical access timeline
physicaltimeline = pd.merge(gpsItems, buildinglogs_results, how='inner', on='device_s')

#Merge Data for wireless access timeline
wirelesstimeline = pd.merge(gpsItems, wireless_results, how='inner', left_on='device_s', right_on='AP_MAC_s')

## Visualize Event Timeline
- using folium maps

In [ ]:
import folium
from folium.plugins import TimestampedGeoJson


campus_map = folium.Map(location=[43.12974644470927, -77.63196881262273], zoom_start=17)

# Sort by 'date' in ascending order
physicaltimeline = physicaltimeline.sort_values(by='eventtime_t')
physicaltimeline = physicaltimeline.reset_index(drop=True)
wirelesstimeline = wirelesstimeline.sort_values(by='timestamp_t')
wirelesstimeline = wirelesstimeline.reset_index(drop=True)

# These functions will be used to scale colors across the timeslice we select
def hex_to_rgb(hex_color):
    # Convert hex color to an RGB tuple
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb_color):
    # Convert an RGB tuple to a hex color
    return '#{:02x}{:02x}{:02x}'.format(*rgb_color)

def interpolate_colors(start_hex, end_hex, steps):
    # Convert start and end colors to RGB
    start_rgb = hex_to_rgb(start_hex)
    end_rgb = hex_to_rgb(end_hex)

    # Calculate the incremental change for each channel
    delta = [(end - start) / (steps - 1) for start, end in zip(start_rgb, end_rgb)]

    # Generate colors
    gradient = []
    for step in range(steps):
        # Calculate intermediate color by adding the increment times the step number
        intermediate_rgb = [
            int(start + delta_value * step) for start, delta_value in zip(start_rgb, delta)
        ]
        gradient.append(rgb_to_hex(intermediate_rgb))
    
    return gradient


lines = []
n = len(physicaltimeline)
#Feel free to pick your favorite colors here, it will scale from the start to the end in equal changes across your timeslice
start_hex = "#83f20c"  # Starting color
end_hex = "#023d24"    # Ending color
gradient = interpolate_colors(start_hex, end_hex, n)

#iterate through our physical badge swipe timeline and plot each progressive point on our map
for index, row in physicaltimeline.iterrows():
    if index == 0:
        lastlat, lastlong, lasttime = row['latitude'], row['longitude'], str(row['eventtime_t'])
        continue
    elif index == 1:
        curlat, curlong, curtime = row['latitude'], row['longitude'], str(row['eventtime_t'])
    else:
        lastlat, lastlong, lasttime = curlat, curlong, curtime
        curlat, curlong, curtime = row['latitude'], row['longitude'], str(row['eventtime_t'])
    
    # This puts the line on the map, with the current and previous waypoints.  Lots of options exist in folium to add additional data to the mapping!
    lines.append({
        "coordinates": [
            [float(lastlong), float(lastlat)],
            [float(curlong), float(curlat)],
        ],
        "message":"Badge Access - " + row['device_s'] + '-' + row['event_s'],
        "dates": [lasttime, curtime],
        "color": gradient[index],
    })

n = len(wirelesstimeline)
start_hex = "#00faed"  # Starting color
end_hex = "#130659"    # Ending color
gradient = interpolate_colors(start_hex, end_hex, n)

#iterate through our network log timeline and plot each progressive point on our map
for index, row in wirelesstimeline.iterrows():
    if index == 0:
        lastlat, lastlong, lasttime = row['latitude'], row['longitude'], str(row['timestamp_t'])
        continue
    elif index == 1:
        curlat, curlong, curtime = row['latitude'], row['longitude'], str(row['timestamp_t'])
    else:
        lastlat, lastlong, lasttime = curlat, curlong, curtime
        curlat, curlong, curtime = row['latitude'], row['longitude'], str(row['timestamp_t'])
    
    # This puts the line on the map, with the current and previous waypoints.  Lots of options exist in folium to add additional data to the mapping!
    lines.append({
        "coordinates": [
            [float(lastlong), float(lastlat)],
            [float(curlong), float(curlat)],
        ],
        "message":"Wireless SSID - " + row['SSID_s'] + " - " + row['EVENT_s'],
        "dates": [lasttime, curtime],
        "color": gradient[index],
    })

# Our features setup overall mapping configurations for the final display.  Lots more options available here as well
features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"],
        },
        "properties": {
            "popup": line["message"],
            "times": line["dates"],
            "style": {
                "color": line["color"],
                "weight": line["weight"] if "weight" in line else 3,
            },
        },
    }
    for line in lines
]

folium.plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT5M",
    add_last_point=True,
).add_to(campus_map)

print("Wireless logs shown in green, building logs shown in blue.  Both get gradually darker over time.")
#display our map!
campus_map

## Export Datasets

In [ ]:
# Save DataFrame to CSV
physicaltimeline.to_csv('physicaltimeline.csv', index=False)
wirelesstimeline.to_csv('wirelesstimeline.csv', index=False)
print("See generated files in file explorer on the left!  You may need to refresh the file viewer.")